# DECISION TREE

In [188]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [189]:
test_clean = pd.read_excel('test_clean.xlsx')
train_clean = pd.read_excel('train_clean.xlsx')
train_label = train_clean['label'].to_numpy()

In [190]:
test_clean.head()


,Unnamed: 0,text,label
0,0,palo alto calif years scorning political proc...,0
1,2,videos nodapl native american leaders vow stay...,0
2,3,dont succeed try different sport tim tebow hei...,1
3,4,42 mins ago 1 views 0 comments 0 likes for tim...,1
4,6,sunday nbcs meet press house minority leader r...,1


In [191]:
print(train_label[:20])

[1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0]


# CARGANDO MODELO ENTRENADO DOC2VEC

In [192]:
model= Doc2Vec.load("d2v-30-30-1.model")

In [193]:
text_vectors = model.docvecs.vectors_docs
print(text_vectors)

[[ 2.029579   -5.0857463   0.57676584 ... -4.890017    2.5237598
  -5.125035  ]
 [ 0.3348972  -2.0161352   3.5392702  ... -0.9070313   0.25327632
  -5.9858203 ]
 [-0.47013155 -1.2324988   3.689975   ... -0.49188724  4.5368466
  -5.399334  ]
 ...
 [ 2.9820414   1.0170153   4.808458   ... -8.428259   -2.179974
   0.72158355]
 [-2.1016812  -0.63795453  2.537954   ... -2.5955384  -0.36465353
  -1.7016034 ]
 [ 1.8858492   1.709495    4.4647775  ...  3.2179825  -0.36528015
  -1.5831916 ]]


# CONFIGURACIÓN Y ENTRENAMIENTO DE DECISION TREE

In [207]:
tree = DecisionTreeClassifier(criterion="entropy", min_samples_split=200, random_state= 99)
tree.fit(text_vectors,train_label)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=200,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=99, splitter='best')

# CARGA Y PROCESAMIENTO DE DATOS DE PRUEBA

In [195]:
test_clean = pd.read_excel('test_clean.xlsx')

test_data = []
vector_test_data = []

for i in test_clean['text']:
    test_data.append(word_tokenize(i))
    
for i in range(len(test_data)):
    vector_test_data.append(model.infer_vector(test_data[i]))

In [196]:
for i in range(len(vector_test_data)):
    vector_test_data[i] = np.array(vector_test_data[i]).reshape(1,30)
    
vector_test_data[:2]

[array([[ 1.1571739 ,  1.6302937 ,  3.0468783 ,  2.3134942 ,  1.2223284 ,
         -0.57258147, -0.412264  , -1.3346804 , -0.84787667, -1.5171411 ,
         -0.64043033,  0.13324675,  0.8086485 ,  1.2461214 , -1.619541  ,
         -0.15584725,  0.19151178,  0.77609354, -0.90637606, -0.10151214,
         -1.1385549 , -2.2717714 , -0.43718415,  1.3122315 ,  0.7387772 ,
         -1.1350175 , -2.708119  , -1.6149026 , -0.18913816, -1.5770895 ]],
       dtype=float32),
 array([[-0.12410868,  1.2437177 ,  2.2509964 ,  0.4117859 ,  0.10370959,
          2.2171578 ,  0.19246188,  1.4827666 ,  1.4080725 ,  1.0275875 ,
         -0.8353295 ,  0.52497154, -0.4933222 , -1.1690015 ,  2.2792559 ,
         -1.7166204 ,  2.2819834 ,  1.3859468 ,  1.120024  ,  0.48577282,
         -1.2304682 , -0.8748603 , -0.5620775 ,  1.4239814 , -0.13019246,
         -0.11638816, -1.9972913 ,  0.3436176 , -1.7005843 , -1.1959227 ]],
       dtype=float32)]

# CLASIFICANDO DATOS DE PRUEBA

In [208]:
test_results = []
for i in vector_test_data:
#     print(classifier.predict(i))
    test_results.append(tree.predict(i)[0])
    
# predict = classifier.predict(vector_test_data[1])
# print(predict)
print(test_results[:20])

[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


# CALCULANDO PRESICIÓN OBTENIDA DEL MODELO

In [209]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(test_results, test_clean['label']))

Accuracy: 0.7185299295774648
